In [ ]:
from Decoder import Decoder
from parser import Parser

In [ ]:
# Configuration of the model: 
numClasses = 10
nms_thres=0.45 # IoU threshold for non-maximal suppression
score_thres=0.01 # threshold for classification scores
top_k=200 # the maximum number of predictions kept per image

In [ ]:
# Load data for prediction
data_dir = "/Users/tranle/mscoco"
predicting_data = "val2017"
# Initialize a parser object
parser = Parser(data_dir, predicting_data)

imgs, labels = parser.load_data() # labels (batch_size, n_boxes, 1 + numClasses + 4)

In [ ]:
# Load trained model
# This method doesnt exist
model = load_model...

In [ ]:
# Load trained weight
weights_path = ''

model.load_weights(weights_path) # TODO

In [ ]:
# Predict object detection
y_pred = model.predict(imgs)

In [ ]:
# Decode prediction result
default = generate_default_boxes(n_layers=n_predictions, 
                                min_scale=min_scale, 
                                max_scale=max_scale, 
                                map_size=prediction_size, 
                                aspect_ratios=aspect_ratios)

# decoder returns (n_boxes, 1 + numClasses + x1 y1 x2 y2)
y_pred_decoded = Decoder(predictions=y_pred, 
                        defaults=defaults, 
                        numClasses=numClasses, 
                        nms_thres=np.float32(0), 
                        score_thres=np.float32(0), 
                        top_k=top_k)

# Get the class_id with highest scores (n_boxes, label, 4)
y_pred_label = y_pred_decoded.prediction_out()

In [ ]:
# Display prediction result
# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()
classes = ['background',
           'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic']

plt.figure(figsize=(20,12))
i = 0
plt.imshow(imgs[i])

current_axis = plt.gca()

for box in labels[i]:
    xmin = box[-4]
    ymin = box[-3]
    xmax = box[-2]
    ymax = box[-1]
    label_id = [i for i,e in enumerate(box(:-4)) if e != 0]# Find the correct class where it is not zero
    label = '{}'.format(classes[int(label_id[0])])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

for box in y_pred_decoded[i]:
    xmin = box[-4]
    ymin = box[-3]
    xmax = box[-2]
    ymax = box[-1]
    label_id = [i for i,e in enumerate(box(:-4)) if e != 0]# Find the correct class where it is not zero
    color = colors[int(label_id[0])]
    label = '{}: {:.2f}'.format(classes[int(label_id[0])], y_pred_label[i][label_id[0]])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})